In [ ]:
# ! pip install pandas great_expectations pandas-profiling cerberus
# ! pip install pandas Pandera 
import pandas as pd
import numpy as np
import pandera as pa
from ydata_profiling import ProfileReport
from pandera import Check, Column, DataFrameSchema



# Creating DataFrames for Importing Tables in Postgres
#### The DataFrames created are for:
- Itinerary
- Cabin Class
- Flight Number
- Airports
- Airlines
- Itinerary Details
- Flight Price

### Creating DataFrame for itinerary
**Primary Key:** `itinerary_id`


In [140]:
# Read the cleaned_price_data csv
flight_price_data =  pd.read_csv('cleaning_output/cleaned_price_data.csv')
flight_price_data.head()

,itinerary_id,cabin_class,one_way_price_candollar,flight_number,origin_airport,destination_airport,departure_date_time,arrival_date_time,stop_count,marketing_airline,operating_airline,is_self_transfer,duration_in_hours
0,18467-2504010745--31679-1-10968-2504011249,economy,477,6340,YYZ,ATL,2025-04-01 07:45:00,2025-04-01 10:17:00,1,Westjet,Delta Air Lines,True,2.53
1,18467-2504010745--31679-1-10968-2504011249,economy,477,6482,ATL,DFW,2025-04-01 11:20:00,2025-04-01 12:49:00,1,Westjet,Delta Air Lines,True,2.48
2,"18390-2504010700--31954,-31825-2-10968-2504011735",economy,253,2205,YTZ,YOW,2025-04-01 07:00:00,2025-04-01 07:59:00,2,Porter Airlines,Porter Airlines Inc,True,0.98
3,18467-2504010615--32385-1-10910-2504011111,premium_economy,581,2662,YYZ,ATL,2025-04-01 06:15:00,2025-04-01 08:49:00,1,Delta Air Lines,Delta Air Lines,True,2.57
4,18467-2504010745--32385-1-10968-2504011249,premium_economy,581,2988,YYZ,ATL,2025-04-01 07:45:00,2025-04-01 10:17:00,1,Delta Air Lines,Delta Air Lines,True,2.53


In [242]:
# Data validation for flight_price_data
# profiling report for flight_price_data using pandas_profiling.
# Generate a profiling report
profile = ProfileReport(flight_price_data, title="Combine Flight Price Profiling Report", explorative=True)

# Save the report to an HTML file
profile.to_file("combine_flight_price_data.html")


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [142]:
# Describe the dataframe
flight_price_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11127 entries, 0 to 11126
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   itinerary_id             11127 non-null  object 
 1   cabin_class              11127 non-null  object 
 2   one_way_price_candollar  11127 non-null  int64  
 3   flight_number            11127 non-null  int64  
 4   origin_airport           11127 non-null  object 
 5   destination_airport      11127 non-null  object 
 6   departure_date_time      11127 non-null  object 
 7   arrival_date_time        11127 non-null  object 
 8   stop_count               11127 non-null  int64  
 9   marketing_airline        11127 non-null  object 
 10  operating_airline        11127 non-null  object 
 11  is_self_transfer         11127 non-null  bool   
 12  duration_in_hours        11127 non-null  float64
dtypes: bool(1), float64(1), int64(3), object(8)
memory usage: 1.0+ MB


In [143]:
# Convert departure_date_time to datetime format
flight_price_data['departure_date_time']=pd.to_datetime(flight_price_data['departure_date_time'], errors='coerce')

In [144]:
# Convert arrival_date_time to datetime format
flight_price_data['arrival_date_time']=pd.to_datetime(flight_price_data['arrival_date_time'], errors='coerce')

In [145]:
flight_price_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11127 entries, 0 to 11126
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   itinerary_id             11127 non-null  object        
 1   cabin_class              11127 non-null  object        
 2   one_way_price_candollar  11127 non-null  int64         
 3   flight_number            11127 non-null  int64         
 4   origin_airport           11127 non-null  object        
 5   destination_airport      11127 non-null  object        
 6   departure_date_time      11127 non-null  datetime64[ns]
 7   arrival_date_time        11127 non-null  datetime64[ns]
 8   stop_count               11127 non-null  int64         
 9   marketing_airline        11127 non-null  object        
 10  operating_airline        11127 non-null  object        
 11  is_self_transfer         11127 non-null  bool          
 12  duration_in_hours        11127 n

##### Extracting unique itinerary_id from  flight_price_data


In [146]:
unique_itineraries = flight_price_data['itinerary_id'].unique()
unique_itineraries

array(['18467-2504010745--31679-1-10968-2504011249',
       '18390-2504010700--31954,-31825-2-10968-2504011735',
       '18467-2504010615--32385-1-10910-2504011111', ...,
       '18467-2509301635--31722-0-10968-2509301857',
       '18467-2509300820--31722-0-10968-2509301042',
       '18467-2509301130--32573-1-10968-2509301841'], dtype=object)

In [147]:
# Create a DataFrame
itinerary_df = pd.DataFrame({"itineraries": unique_itineraries})
itinerary_df
# Add a new column with sequential labels ('it1', 'it2', ...)
itinerary_df['itinerary_id'] = [f"it{i+1}" for i in range(len(itinerary_df))]
itinerary_df = itinerary_df[['itinerary_id','itineraries']]
itinerary_df

,itinerary_id,itineraries
0,it1,18467-2504010745--31679-1-10968-2504011249
1,it2,"18390-2504010700--31954,-31825-2-10968-2504011735"
2,it3,18467-2504010615--32385-1-10910-2504011111
3,it4,18467-2504010745--32385-1-10968-2504011249
4,it5,18467-2504011220--32385-1-10910-2504011753
...,...,...
6322,it6323,18467-2509301320--32385-1-10968-2509301833
6323,it6324,18467-2509301130--32385-1-10968-2509301647
6324,it6325,18467-2509301635--31722-0-10968-2509301857
6325,it6326,18467-2509300820--31722-0-10968-2509301042


In [148]:
itinerary_df.to_csv('postgrs_tables/itinerary.csv', index=False)

In [149]:
itinerary_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6327 entries, 0 to 6326
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   itinerary_id  6327 non-null   object
 1   itineraries   6327 non-null   object
dtypes: object(2)
memory usage: 99.0+ KB


# Creating DataFrame for Cabin Class
**Primary Key:** `cabin_class_id`

### Extracting unique cabin classes from the cabin_class column of flight_price_data

In [150]:
unique_cabin_classes = flight_price_data['cabin_class'].unique()
unique_cabin_classes


array(['economy', 'premium_economy', 'business', 'first'], dtype=object)

In [151]:
# Create a DataFrame
unique_cabin_classes_df = pd.DataFrame({"cabin_class": unique_cabin_classes})
unique_cabin_classes_df
# Add a new column with sequential labels ('cc1', 'cc2', ...)
unique_cabin_classes_df['cabin_class_id'] = [f"cc{i+1}" for i in range(len(unique_cabin_classes_df))]
unique_cabin_classes_df = unique_cabin_classes_df[['cabin_class_id','cabin_class']]
unique_cabin_classes_df

,cabin_class_id,cabin_class
0,cc1,economy
1,cc2,premium_economy
2,cc3,business
3,cc4,first


In [152]:
# Validation for cabin_class
# we have unique_cabin_classes_df ready
# define schema
schema = pa.DataFrameSchema({
    "cabin_class_id": pa.Column(str, checks=pa.Check.str_length(min_value=3, max_value=4)),
    "cabin_class": pa.Column(str, checks=pa.Check.str_length(min_value=5, max_value=15))
      })

# Validate the dataframe
try:
    schema.validate(unique_cabin_classes_df)
    print("Data is valid!")
    unique_cabin_classes_df = schema(unique_cabin_classes_df)
except pa.errors.SchemaError as e:
    print(f"Data validation error: {e}")

Data is valid!


In [153]:
unique_cabin_classes_df.to_csv('postgrs_tables/cabin_class.csv', index=False)

# Creating DataFrame for Airports
**Primary Key:** `airport_id`

In [154]:
# Read in cleaned_extracted_airports csv file
airports_df = pd.read_csv('cleaning_output/cleaned_extracted_airports.csv')
airports_df

,name,iata code,country,city,latitude,longitude,timezone,departures
0,Billy Bishop Toronto City Airport,YTZ,CA,Toronto,43.62974,-79.39828,EST,752
1,Kitchener/Waterloo Airport,YKF,CA,Breslau,43.45747,-80.38593,EST,454
2,Montreal-Pierre Elliott Trudeau International ...,YUL,CA,Montreal,45.46106,-73.75019,EST,35190
3,Ottawa Macdonald-Cartier International Airport,YOW,CA,Ottawa,45.32250,-75.66917,EST,10293
4,Toronto Pearson International Airport,YYZ,CA,Toronto,43.68066,-79.61286,EST,72355
5,Chicago O'Hare International Airport,ORD,US,Chicago,41.97959,-87.90446,CST,177167
6,Dallas/Fort Worth International Airport,DFW,US,Dallas-Ft Worth,32.89595,-97.03720,CST,172447
7,Denver International Airport,DEN,US,Denver,39.85891,-104.67326,MST,100565
8,Hartsfield-Jackson Atlanta International Airport,ATL,US,Atlanta,33.64099,-84.42265,EST,168164
9,Los Angeles International Airport,LAX,US,Los Angeles,33.94251,-118.40897,PST,112571


In [155]:
# Add airline_id as an index-based unique identifier
airports_df['airport_id'] = [f"ap{i+1}" for i in range(len(airports_df))]
airports_df

,name,iata code,country,city,latitude,longitude,timezone,departures,airport_id
0,Billy Bishop Toronto City Airport,YTZ,CA,Toronto,43.62974,-79.39828,EST,752,ap1
1,Kitchener/Waterloo Airport,YKF,CA,Breslau,43.45747,-80.38593,EST,454,ap2
2,Montreal-Pierre Elliott Trudeau International ...,YUL,CA,Montreal,45.46106,-73.75019,EST,35190,ap3
3,Ottawa Macdonald-Cartier International Airport,YOW,CA,Ottawa,45.32250,-75.66917,EST,10293,ap4
4,Toronto Pearson International Airport,YYZ,CA,Toronto,43.68066,-79.61286,EST,72355,ap5
5,Chicago O'Hare International Airport,ORD,US,Chicago,41.97959,-87.90446,CST,177167,ap6
6,Dallas/Fort Worth International Airport,DFW,US,Dallas-Ft Worth,32.89595,-97.03720,CST,172447,ap7
7,Denver International Airport,DEN,US,Denver,39.85891,-104.67326,MST,100565,ap8
8,Hartsfield-Jackson Atlanta International Airport,ATL,US,Atlanta,33.64099,-84.42265,EST,168164,ap9
9,Los Angeles International Airport,LAX,US,Los Angeles,33.94251,-118.40897,PST,112571,ap10


In [156]:
airports_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        10 non-null     object 
 1   iata code   10 non-null     object 
 2   country     10 non-null     object 
 3   city        10 non-null     object 
 4   latitude    10 non-null     float64
 5   longitude   10 non-null     float64
 6   timezone    10 non-null     object 
 7   departures  10 non-null     int64  
 8   airport_id  10 non-null     object 
dtypes: float64(2), int64(1), object(6)
memory usage: 848.0+ bytes


# Creating DataFrame for Airlines
**Primary Key:** `airline_id`

In [157]:
# Read in the airlines data
airlines_df =  pd.read_csv('cleaning_output/cleaned_extracted_airlines.csv')

In [158]:
airlines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   airline_name          17 non-null     object 
 1   country_code          17 non-null     object 
 2   iata                  17 non-null     object 
 3   iosa_registered       17 non-null     bool   
 4   iosa_expiry           15 non-null     object 
 5   is_airline_passenger  17 non-null     bool   
 6   total_aircrafts       17 non-null     int64  
 7   average_fleet_age     17 non-null     float64
 8   accidents_last_5y     17 non-null     int64  
dtypes: bool(2), float64(1), int64(2), object(4)
memory usage: 1.1+ KB


In [159]:
# Convert to datetime
airlines_df['iosa_expiry'] = pd.to_datetime(airlines_df['iosa_expiry'], errors='coerce')
# Fill missing value with NAT 
airlines_df['iosa_expiry'] = airlines_df['iosa_expiry'].fillna(pd.NaT)

In [160]:
airlines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   airline_name          17 non-null     object             
 1   country_code          17 non-null     object             
 2   iata                  17 non-null     object             
 3   iosa_registered       17 non-null     bool               
 4   iosa_expiry           15 non-null     datetime64[ns, UTC]
 5   is_airline_passenger  17 non-null     bool               
 6   total_aircrafts       17 non-null     int64              
 7   average_fleet_age     17 non-null     float64            
 8   accidents_last_5y     17 non-null     int64              
dtypes: bool(2), datetime64[ns, UTC](1), float64(1), int64(2), object(3)
memory usage: 1.1+ KB


In [161]:
# Add airline_id as an index-based unique identifier
airlines_df['airline_id'] = [f"al{i+1}" for i in range(len(airlines_df))]

airlines_df

,airline_name,country_code,iata,iosa_registered,iosa_expiry,is_airline_passenger,total_aircrafts,average_fleet_age,accidents_last_5y,airline_id
0,Alaska Airlines,US,AS,True,2025-01-16 00:00:00+00:00,True,158,8.1,1,al1
1,American Airlines,US,AA,True,2025-07-29 00:00:00+00:00,True,684,10.2,26,al2
2,Air Canada,CA,AC,True,2024-04-18 00:00:00+00:00,True,98,11.7,0,al3
3,Delta Air Lines,US,DL,True,2024-10-17 00:00:00+00:00,True,591,12.5,22,al4
4,Envoy Air,US,MQ,True,2025-11-18 00:00:00+00:00,True,161,9.6,0,al5
5,Frontier Airlines,US,F9,True,2024-01-03 00:00:00+00:00,True,83,3.0,4,al6
6,GOL Linhas Aereas Inteligentes,BR,G3,True,2025-11-23 00:00:00+00:00,True,103,10.7,0,al7
7,Jazz Aviation,CA,QK,True,2024-10-21 00:00:00+00:00,True,85,13.8,3,al8
8,Lufthansa,DE,LH,True,2026-05-12 00:00:00+00:00,True,131,8.9,1,al9
9,Porter Airlines Inc,CA,P3,True,NaT,True,1,8.0,0,al10


In [162]:
airlines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   airline_name          17 non-null     object             
 1   country_code          17 non-null     object             
 2   iata                  17 non-null     object             
 3   iosa_registered       17 non-null     bool               
 4   iosa_expiry           15 non-null     datetime64[ns, UTC]
 5   is_airline_passenger  17 non-null     bool               
 6   total_aircrafts       17 non-null     int64              
 7   average_fleet_age     17 non-null     float64            
 8   accidents_last_5y     17 non-null     int64              
 9   airline_id            17 non-null     object             
dtypes: bool(2), datetime64[ns, UTC](1), float64(1), int64(2), object(4)
memory usage: 1.2+ KB


In [163]:
# Reorder the columns
airlines_df=airlines_df[['airline_id','airline_name','iata','country_code','is_airline_passenger','total_aircrafts','average_fleet_age','accidents_last_5y','iosa_registered','iosa_expiry']]
airlines_df.head()

,airline_id,airline_name,iata,country_code,is_airline_passenger,total_aircrafts,average_fleet_age,accidents_last_5y,iosa_registered,iosa_expiry
0,al1,Alaska Airlines,AS,US,True,158,8.1,1,True,2025-01-16 00:00:00+00:00
1,al2,American Airlines,AA,US,True,684,10.2,26,True,2025-07-29 00:00:00+00:00
2,al3,Air Canada,AC,CA,True,98,11.7,0,True,2024-04-18 00:00:00+00:00
3,al4,Delta Air Lines,DL,US,True,591,12.5,22,True,2024-10-17 00:00:00+00:00
4,al5,Envoy Air,MQ,US,True,161,9.6,0,True,2025-11-18 00:00:00+00:00


# Creating DataFrame for Country Code
**Primary Key:** `country_id`


In [164]:
# Read in unique country code from airlines dataframe
unique_country_code = airlines_df['country_code'].unique()
unique_country_code

array(['US', 'CA', 'BR', 'DE'], dtype=object)

In [165]:
# Create a DataFrame
unique_country_code = pd.DataFrame({"country_code": unique_country_code})
unique_country_code
# Add a new column with sequential labels ('cc1', 'cc2', ...)
unique_country_code['country_code_id'] = [f"coc{i+1}" for i in range(len(unique_country_code))]
unique_country_code = unique_country_code[['country_code_id','country_code']]
unique_country_code

,country_code_id,country_code
0,coc1,US
1,coc2,CA
2,coc3,BR
3,coc4,DE


In [204]:
# dataframe to validate 
# we have unique_country_code ready
# define schema
schema = pa.DataFrameSchema({
    "country_code_id": pa.Column(str,unique=True),
    "country_code": pa.Column(str)   
      })

# Validate the dataframe
try:
    schema.validate(unique_country_code)
    print("Data is valid!")
    unique_country_code = schema(unique_country_code)
except pa.errors.SchemaError as e:
    print(f"Data validation error: {e}")

Data is valid!


In [205]:
unique_country_code.to_csv('postgrs_tables/country_code.csv',index=False)

# Creating DataFrame for Flight Numbers
**Primary Key:** `flight_id`

In [168]:
# Get unique flight numbers from flight price data
unique_flights = flight_price_data['flight_number'].unique()
unique_flights

array([6340, 6482, 2205, 2662, 2988,  414, 2835, 3993, 1111, 1169, 8019,
       8130, 4731, 2257, 3524,  481, 3608,  550,  611,  921, 1538,  307,
       1109, 2714,  818, 8135, 6768, 6499, 2499, 3606, 7789, 6498,  374,
       8533, 3449, 5578, 2290, 1031, 1033, 4476, 4408, 4854, 4990, 5521,
        894, 8813, 5143, 4888, 5634, 3610, 2281,  598, 6536, 5141, 5186,
       8901, 3658, 2259, 2217, 2710, 3604,  436, 7106, 8720, 8742, 2249,
        505, 4235, 7129, 4902, 3371, 6502,  420, 7968, 5137, 5481, 2219,
       2461,  920,  961, 1993, 8156, 8169, 1491, 3499,  490, 1297, 3605,
        793, 1290, 1555, 8080, 1029,  507,  503, 3132, 8903, 1071, 1053,
       3078, 5588, 8909, 4770, 2346,   31, 4530, 8779, 6999, 6479, 6931,
       8278, 2421, 8078, 3170, 3297, 1239, 2999,  501, 1893, 3102, 1401,
       1914, 2862, 2868, 1213, 2247, 1961, 1390, 1642,  416, 2483, 2473,
       2463, 2459, 2485, 4238, 2156, 1841, 2536, 1362, 8307, 3444, 3489,
        516, 8085, 2364, 8694, 1193, 1371, 1856, 86

In [169]:

# Create a DataFrame properly
unique_flights_df = pd.DataFrame({"flight_number": unique_flights})

# Add a new column with sequential labels ('fl1', 'fl2', ...)
unique_flights_df['flight_id'] = [f"{i+1}" for i in range(len(unique_flights_df))]
# Reorder columns
unique_flights_df = unique_flights_df[['flight_id','flight_number']]

#Convert it into int type
unique_flights_df['flight_id'] = unique_flights_df['flight_id'].astype('int64')

In [170]:
unique_flights_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195 entries, 0 to 194
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   flight_id      195 non-null    int64
 1   flight_number  195 non-null    int64
dtypes: int64(2)
memory usage: 3.2 KB


In [207]:
# dataframe to validate 
# we have unique_flights_df ready
# define schema
schema = pa.DataFrameSchema({
    "flight_id": pa.Column(int,unique=True),
    "flight_number": pa.Column(int, unique=True)   
      })

# Validate the dataframe
try:
    schema.validate(unique_flights_df)
    print("Data is valid!")
    unique_flights_df = schema(unique_flights_df)
except pa.errors.SchemaError as e:
    print(f"Data validation error: {e}")

Data is valid!


In [208]:
unique_flights_df.to_csv('postgrs_tables/unique_flights.csv',index=False)

# Merge airport with country_code

In [172]:
airport_data = pd.merge(airports_df,unique_country_code,left_on='country', right_on='country_code', how='left')
airport_data.head()

,name,iata code,country,city,latitude,longitude,timezone,departures,airport_id,country_code_id,country_code
0,Billy Bishop Toronto City Airport,YTZ,CA,Toronto,43.62974,-79.39828,EST,752,ap1,coc2,CA
1,Kitchener/Waterloo Airport,YKF,CA,Breslau,43.45747,-80.38593,EST,454,ap2,coc2,CA
2,Montreal-Pierre Elliott Trudeau International ...,YUL,CA,Montreal,45.46106,-73.75019,EST,35190,ap3,coc2,CA
3,Ottawa Macdonald-Cartier International Airport,YOW,CA,Ottawa,45.32250,-75.66917,EST,10293,ap4,coc2,CA
4,Toronto Pearson International Airport,YYZ,CA,Toronto,43.68066,-79.61286,EST,72355,ap5,coc2,CA


In [173]:
# Drop unwanted columnss
airport_data.drop(columns=['country','country_code'], errors='ignore', inplace=True)

In [174]:
airport_data.head()

,name,iata code,city,latitude,longitude,timezone,departures,airport_id,country_code_id
0,Billy Bishop Toronto City Airport,YTZ,Toronto,43.62974,-79.39828,EST,752,ap1,coc2
1,Kitchener/Waterloo Airport,YKF,Breslau,43.45747,-80.38593,EST,454,ap2,coc2
2,Montreal-Pierre Elliott Trudeau International ...,YUL,Montreal,45.46106,-73.75019,EST,35190,ap3,coc2
3,Ottawa Macdonald-Cartier International Airport,YOW,Ottawa,45.32250,-75.66917,EST,10293,ap4,coc2
4,Toronto Pearson International Airport,YYZ,Toronto,43.68066,-79.61286,EST,72355,ap5,coc2


In [175]:
airport_data.info()
# Getting error for Data Validation for datatype
# Convert the 'departures' column to float64
airport_data['departures'] = airport_data['departures'].astype(float)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             10 non-null     object 
 1   iata code        10 non-null     object 
 2   city             10 non-null     object 
 3   latitude         10 non-null     float64
 4   longitude        10 non-null     float64
 5   timezone         10 non-null     object 
 6   departures       10 non-null     int64  
 7   airport_id       10 non-null     object 
 8   country_code_id  10 non-null     object 
dtypes: float64(2), int64(1), object(6)
memory usage: 848.0+ bytes


In [209]:
# Dataframe to validate 
# we have filtered_airports_df ready
# define schema
schema = pa.DataFrameSchema({
    "name": pa.Column(str), 
    "iata code": pa.Column(str,unique=True),    
    "country_code_id": pa.Column(str,checks=pa.Check.str_length(max_value=4)),
    "city": pa.Column(str, checks=Check.isin(['Toronto','Breslau','Montreal','Ottawa','Chicago',
                                             'Dallas-Ft Worth','Denver','Atlanta','Atlanta',
                                             'Los Angeles'])),
    "latitude": pa.Column(float, checks=[Check.ge(-90.0), Check.le(90.0)]),  
    "longitude": pa.Column(float, checks=[Check.ge(-180.0), Check.le(180.0)]),  
    "timezone": pa.Column(str, checks=pa.Check.str_length(3)),
    "departures": pa.Column(np.float64, nullable=True)    
      })

# Validate the dataframe
try:
    schema.validate(airport_data)
    print("Data is valid!")
    airports_df = schema(airport_data)
except pa.errors.SchemaError as e:
    print(f"Data validation error: {e}")

Data is valid!


In [177]:
airport_data.to_csv('postgrs_tables/mapped_airport_data.csv', index=False)

# Merge airlines with country code 

In [178]:
airline_data = pd.merge(airlines_df,unique_country_code,left_on='country_code', right_on='country_code', how='left')
airline_data.head()

,airline_id,airline_name,iata,country_code,is_airline_passenger,total_aircrafts,average_fleet_age,accidents_last_5y,iosa_registered,iosa_expiry,country_code_id
0,al1,Alaska Airlines,AS,US,True,158,8.1,1,True,2025-01-16 00:00:00+00:00,coc1
1,al2,American Airlines,AA,US,True,684,10.2,26,True,2025-07-29 00:00:00+00:00,coc1
2,al3,Air Canada,AC,CA,True,98,11.7,0,True,2024-04-18 00:00:00+00:00,coc2
3,al4,Delta Air Lines,DL,US,True,591,12.5,22,True,2024-10-17 00:00:00+00:00,coc1
4,al5,Envoy Air,MQ,US,True,161,9.6,0,True,2025-11-18 00:00:00+00:00,coc1


In [179]:
# Drop unwanted columns
airline_data.drop(columns=['country_code'] ,errors='ignore', inplace=True )

In [180]:
airline_data.head()

,airline_id,airline_name,iata,is_airline_passenger,total_aircrafts,average_fleet_age,accidents_last_5y,iosa_registered,iosa_expiry,country_code_id
0,al1,Alaska Airlines,AS,True,158,8.1,1,True,2025-01-16 00:00:00+00:00,coc1
1,al2,American Airlines,AA,True,684,10.2,26,True,2025-07-29 00:00:00+00:00,coc1
2,al3,Air Canada,AC,True,98,11.7,0,True,2024-04-18 00:00:00+00:00,coc2
3,al4,Delta Air Lines,DL,True,591,12.5,22,True,2024-10-17 00:00:00+00:00,coc1
4,al5,Envoy Air,MQ,True,161,9.6,0,True,2025-11-18 00:00:00+00:00,coc1


In [181]:
airline_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype              
---  ------                --------------  -----              
 0   airline_id            17 non-null     object             
 1   airline_name          17 non-null     object             
 2   iata                  17 non-null     object             
 3   is_airline_passenger  17 non-null     bool               
 4   total_aircrafts       17 non-null     int64              
 5   average_fleet_age     17 non-null     float64            
 6   accidents_last_5y     17 non-null     int64              
 7   iosa_registered       17 non-null     bool               
 8   iosa_expiry           15 non-null     datetime64[ns, UTC]
 9   country_code_id       17 non-null     object             
dtypes: bool(2), datetime64[ns, UTC](1), float64(1), int64(2), object(4)
memory usage: 1.2+ KB


In [218]:
# Convert 'iosa_expiry' to naive datetime (remove timezone)
airline_data['iosa_expiry'] = airline_data['iosa_expiry'].dt.tz_convert(None)

# Convert to datetime64[ns]
airline_data['iosa_expiry'] = airline_data['iosa_expiry'].astype('datetime64[ns]')


In [219]:
# Airlines Dataframe ready for data validation
# dataframe to validate 
# we have airlines_df ready
# define schema
schema = pa.DataFrameSchema({
    "airline_id": pa.Column(str,unique=True),
    "airline_name": pa.Column(str,unique=True), 
    "iata": pa.Column(str,checks=pa.Check.str_length(max_value=2)),
    "country_code_id": pa.Column(str,checks=pa.Check.str_length(max_value=4)),
    "is_airline_passenger": pa.Column(bool),  
    "total_aircrafts": pa.Column(int),  
    "average_fleet_age": pa.Column(np.float64),
    "accidents_last_5y": pa.Column(int),
    "iosa_registered": pa.Column(bool),
    "iosa_expiry": pa.Column(pd.Timestamp,nullable=True)
                })

# Validate the dataframe
try:
    schema.validate(airline_data)
    print("Data is valid!")
    airline_data = schema(airline_data)
except pa.errors.SchemaError as e:
    print(f"Data validation error: {e}")

Data is valid!


In [220]:
airline_data.to_csv('postgrs_tables/mapped_airline_data.csv', index=False)

# Creating dataframe for itinerary_details. Here flight number is foreign key to flightid, origin and destination airports are foreign key to airport id , itinerary is foreign key to itinerary_id.

In [183]:
itinerary_details_df = flight_price_data[['itinerary_id','departure_date_time','arrival_date_time','origin_airport','destination_airport','is_self_transfer','flight_number']]
itinerary_details_df.head()

,itinerary_id,departure_date_time,arrival_date_time,origin_airport,destination_airport,is_self_transfer,flight_number
0,18467-2504010745--31679-1-10968-2504011249,2025-04-01 07:45:00,2025-04-01 10:17:00,YYZ,ATL,True,6340
1,18467-2504010745--31679-1-10968-2504011249,2025-04-01 11:20:00,2025-04-01 12:49:00,ATL,DFW,True,6482
2,"18390-2504010700--31954,-31825-2-10968-2504011735",2025-04-01 07:00:00,2025-04-01 07:59:00,YTZ,YOW,True,2205
3,18467-2504010615--32385-1-10910-2504011111,2025-04-01 06:15:00,2025-04-01 08:49:00,YYZ,ATL,True,2662
4,18467-2504010745--32385-1-10968-2504011249,2025-04-01 07:45:00,2025-04-01 10:17:00,YYZ,ATL,True,2988


##### Reordering columns

In [184]:

itinerary_details_df=itinerary_details_df[['itinerary_id','flight_number','origin_airport','destination_airport','departure_date_time','arrival_date_time','is_self_transfer']]

itinerary_details_df.head()

,itinerary_id,flight_number,origin_airport,destination_airport,departure_date_time,arrival_date_time,is_self_transfer
0,18467-2504010745--31679-1-10968-2504011249,6340,YYZ,ATL,2025-04-01 07:45:00,2025-04-01 10:17:00,True
1,18467-2504010745--31679-1-10968-2504011249,6482,ATL,DFW,2025-04-01 11:20:00,2025-04-01 12:49:00,True
2,"18390-2504010700--31954,-31825-2-10968-2504011735",2205,YTZ,YOW,2025-04-01 07:00:00,2025-04-01 07:59:00,True
3,18467-2504010615--32385-1-10910-2504011111,2662,YYZ,ATL,2025-04-01 06:15:00,2025-04-01 08:49:00,True
4,18467-2504010745--32385-1-10968-2504011249,2988,YYZ,ATL,2025-04-01 07:45:00,2025-04-01 10:17:00,True


In [185]:
# Convert departure_date_time to datetime format
itinerary_details_df['departure_date_time']=pd.to_datetime(itinerary_details_df['departure_date_time'], errors='coerce')

In [186]:
# Convert arrival_date_time to datetime format
itinerary_details_df['arrival_date_time']=pd.to_datetime(itinerary_details_df['arrival_date_time'], errors='coerce')

In [187]:
itinerary_details_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11127 entries, 0 to 11126
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   itinerary_id         11127 non-null  object        
 1   flight_number        11127 non-null  int64         
 2   origin_airport       11127 non-null  object        
 3   destination_airport  11127 non-null  object        
 4   departure_date_time  11127 non-null  datetime64[ns]
 5   arrival_date_time    11127 non-null  datetime64[ns]
 6   is_self_transfer     11127 non-null  bool          
dtypes: bool(1), datetime64[ns](2), int64(1), object(3)
memory usage: 532.6+ KB


# Merge itinerary_details with flight_number, itinerary_id, origin_airport, destination_airport

##### Merge itinerary_details with flight_number dataframe.

In [188]:
itinerary_details = pd.merge(itinerary_details_df,unique_flights_df, how='left', on='flight_number')
itinerary_details.head()

,itinerary_id,flight_number,origin_airport,destination_airport,departure_date_time,arrival_date_time,is_self_transfer,flight_id
0,18467-2504010745--31679-1-10968-2504011249,6340,YYZ,ATL,2025-04-01 07:45:00,2025-04-01 10:17:00,True,1
1,18467-2504010745--31679-1-10968-2504011249,6482,ATL,DFW,2025-04-01 11:20:00,2025-04-01 12:49:00,True,2
2,"18390-2504010700--31954,-31825-2-10968-2504011735",2205,YTZ,YOW,2025-04-01 07:00:00,2025-04-01 07:59:00,True,3
3,18467-2504010615--32385-1-10910-2504011111,2662,YYZ,ATL,2025-04-01 06:15:00,2025-04-01 08:49:00,True,4
4,18467-2504010745--32385-1-10968-2504011249,2988,YYZ,ATL,2025-04-01 07:45:00,2025-04-01 10:17:00,True,5


##### Merge with itinerary dataframe.

In [189]:
itineraries_df = pd.merge(itinerary_details,itinerary_df, how='left', left_on='itinerary_id', right_on='itineraries').rename(columns={'itinerary_id_y':'itinerary_id'})
itineraries_df.head()

,itinerary_id_x,flight_number,origin_airport,destination_airport,departure_date_time,arrival_date_time,is_self_transfer,flight_id,itinerary_id,itineraries
0,18467-2504010745--31679-1-10968-2504011249,6340,YYZ,ATL,2025-04-01 07:45:00,2025-04-01 10:17:00,True,1,it1,18467-2504010745--31679-1-10968-2504011249
1,18467-2504010745--31679-1-10968-2504011249,6482,ATL,DFW,2025-04-01 11:20:00,2025-04-01 12:49:00,True,2,it1,18467-2504010745--31679-1-10968-2504011249
2,"18390-2504010700--31954,-31825-2-10968-2504011735",2205,YTZ,YOW,2025-04-01 07:00:00,2025-04-01 07:59:00,True,3,it2,"18390-2504010700--31954,-31825-2-10968-2504011735"
3,18467-2504010615--32385-1-10910-2504011111,2662,YYZ,ATL,2025-04-01 06:15:00,2025-04-01 08:49:00,True,4,it3,18467-2504010615--32385-1-10910-2504011111
4,18467-2504010745--32385-1-10968-2504011249,2988,YYZ,ATL,2025-04-01 07:45:00,2025-04-01 10:17:00,True,5,it4,18467-2504010745--32385-1-10968-2504011249


##### Merge with airport dataframe

In [190]:
# Merge for origin_airport_id
itinerary_details_merged_df = itineraries_df.merge(
    airport_data[['airport_id', 'iata code']], 
    left_on='origin_airport', 
    right_on='iata code', 
    how='left'
).rename(columns={'airport_id': 'origin_airport_id'})

# Drop redundant column
itinerary_details_merged_df.drop(columns=['iata code'], inplace=True)

itinerary_details_merged_df = itinerary_details_merged_df.merge(
    airport_data[['airport_id', 'iata code']], 
    left_on='destination_airport', 
    right_on='iata code', 
    how='left'
).rename(columns={'airport_id': 'destination_airport_id'})

# Drop redundant column
itinerary_details_merged_df.drop(columns=['iata code'], inplace=True)

# Display final DataFrame
itinerary_details_merged_df.head()

,itinerary_id_x,flight_number,origin_airport,destination_airport,departure_date_time,arrival_date_time,is_self_transfer,flight_id,itinerary_id,itineraries,origin_airport_id,destination_airport_id
0,18467-2504010745--31679-1-10968-2504011249,6340,YYZ,ATL,2025-04-01 07:45:00,2025-04-01 10:17:00,True,1,it1,18467-2504010745--31679-1-10968-2504011249,ap5,ap9
1,18467-2504010745--31679-1-10968-2504011249,6482,ATL,DFW,2025-04-01 11:20:00,2025-04-01 12:49:00,True,2,it1,18467-2504010745--31679-1-10968-2504011249,ap9,ap7
2,"18390-2504010700--31954,-31825-2-10968-2504011735",2205,YTZ,YOW,2025-04-01 07:00:00,2025-04-01 07:59:00,True,3,it2,"18390-2504010700--31954,-31825-2-10968-2504011735",ap1,ap4
3,18467-2504010615--32385-1-10910-2504011111,2662,YYZ,ATL,2025-04-01 06:15:00,2025-04-01 08:49:00,True,4,it3,18467-2504010615--32385-1-10910-2504011111,ap5,ap9
4,18467-2504010745--32385-1-10968-2504011249,2988,YYZ,ATL,2025-04-01 07:45:00,2025-04-01 10:17:00,True,5,it4,18467-2504010745--32385-1-10968-2504011249,ap5,ap9


##### Drop unwanted columns

In [191]:
itinerary_details_merged_df.drop(columns=['itinerary_id_x','flight_number','itineraries','origin_airport','destination_airport'],errors='ignore', inplace=True)

In [221]:
itinerary_details_merged_df.columns

Index(['departure_date_time', 'arrival_date_time', 'is_self_transfer',
       'flight_id', 'itinerary_id', 'origin_airport_id',
       'destination_airport_id'],
      dtype='object')

In [192]:
itinerary_details_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11127 entries, 0 to 11126
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   departure_date_time     11127 non-null  datetime64[ns]
 1   arrival_date_time       11127 non-null  datetime64[ns]
 2   is_self_transfer        11127 non-null  bool          
 3   flight_id               11127 non-null  int64         
 4   itinerary_id            11127 non-null  object        
 5   origin_airport_id       11127 non-null  object        
 6   destination_airport_id  11127 non-null  object        
dtypes: bool(1), datetime64[ns](2), int64(1), object(3)
memory usage: 532.6+ KB


In [244]:
# Dataframe itinerary_details for validation  
# Defining schema
schema = DataFrameSchema({
    "departure_date_time": Column(pa.DateTime, nullable=False),
    "arrival_date_time": Column(pa.DateTime, nullable=False),
    "is_self_transfer": Column(bool, nullable=False),
    "flight_id": Column(int, nullable=False),
    "itinerary_id": Column(str, nullable=False),
    "origin_airport_id": Column(str, nullable=False),
    "destination_airport_id": Column(str, nullable=False),
}, checks=[
    # Custom check to make sure origin and destination airports are not the same
    Check(lambda df: df["origin_airport_id"] != df["destination_airport_id"],
          element_wise=False,
          error="Origin and destination airports must not be the same")
])

# Validate the DataFrame
try:
    itinerary_details_merged_df = schema.validate(itinerary_details_merged_df)
    print("Data is valid!")
except pa.errors.SchemaErrors as e:
    print("Data validation failed!")
    print(e.failure_cases)

Data is valid!


In [193]:
itinerary_details_merged_df.to_csv('postgrs_tables/map_itinerary_details.csv', index=False)

# Creating Flight Price DataFrame
**Foreign Keys:**
- `itinerary` (foreign key of the Itinerary DataFrame)
- `marketing_airline` and `operating_airline` (foreign keys of the Airline DataFrame)
- `cabin_class` (foreign key of the Cabin Class DataFrame)


In [194]:
flight_price_df = flight_price_data[['itinerary_id','marketing_airline','operating_airline','cabin_class','stop_count','duration_in_hours','one_way_price_candollar']]
flight_price_df.head()

,itinerary_id,marketing_airline,operating_airline,cabin_class,stop_count,duration_in_hours,one_way_price_candollar
0,18467-2504010745--31679-1-10968-2504011249,Westjet,Delta Air Lines,economy,1,2.53,477
1,18467-2504010745--31679-1-10968-2504011249,Westjet,Delta Air Lines,economy,1,2.48,477
2,"18390-2504010700--31954,-31825-2-10968-2504011735",Porter Airlines,Porter Airlines Inc,economy,2,0.98,253
3,18467-2504010615--32385-1-10910-2504011111,Delta Air Lines,Delta Air Lines,premium_economy,1,2.57,581
4,18467-2504010745--32385-1-10968-2504011249,Delta Air Lines,Delta Air Lines,premium_economy,1,2.53,581


In [195]:
flight_price_df=flight_price_df[['itinerary_id','cabin_class','stop_count','duration_in_hours','one_way_price_candollar','marketing_airline','operating_airline']]
flight_price_df.head()

,itinerary_id,cabin_class,stop_count,duration_in_hours,one_way_price_candollar,marketing_airline,operating_airline
0,18467-2504010745--31679-1-10968-2504011249,economy,1,2.53,477,Westjet,Delta Air Lines
1,18467-2504010745--31679-1-10968-2504011249,economy,1,2.48,477,Westjet,Delta Air Lines
2,"18390-2504010700--31954,-31825-2-10968-2504011735",economy,2,0.98,253,Porter Airlines,Porter Airlines Inc
3,18467-2504010615--32385-1-10910-2504011111,premium_economy,1,2.57,581,Delta Air Lines,Delta Air Lines
4,18467-2504010745--32385-1-10968-2504011249,premium_economy,1,2.53,581,Delta Air Lines,Delta Air Lines


# Merging Flight Price DataFrame
## With Cabin Class, Operating Airline, Marketing Airline, and Itinerary

##### Merge flight_price_df with cabin_class

In [196]:
class_flight_price_df = pd.merge(flight_price_df,unique_cabin_classes_df, on='cabin_class', how='left')
class_flight_price_df.head()

,itinerary_id,cabin_class,stop_count,duration_in_hours,one_way_price_candollar,marketing_airline,operating_airline,cabin_class_id
0,18467-2504010745--31679-1-10968-2504011249,economy,1,2.53,477,Westjet,Delta Air Lines,cc1
1,18467-2504010745--31679-1-10968-2504011249,economy,1,2.48,477,Westjet,Delta Air Lines,cc1
2,"18390-2504010700--31954,-31825-2-10968-2504011735",economy,2,0.98,253,Porter Airlines,Porter Airlines Inc,cc1
3,18467-2504010615--32385-1-10910-2504011111,premium_economy,1,2.57,581,Delta Air Lines,Delta Air Lines,cc2
4,18467-2504010745--32385-1-10968-2504011249,premium_economy,1,2.53,581,Delta Air Lines,Delta Air Lines,cc2


In [197]:
class_flight_price_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11127 entries, 0 to 11126
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   itinerary_id             11127 non-null  object 
 1   cabin_class              11127 non-null  object 
 2   stop_count               11127 non-null  int64  
 3   duration_in_hours        11127 non-null  float64
 4   one_way_price_candollar  11127 non-null  int64  
 5   marketing_airline        11127 non-null  object 
 6   operating_airline        11127 non-null  object 
 7   cabin_class_id           11127 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 695.6+ KB


In [198]:
# Drop unwanted columns
class_flight_price_df.drop(columns='cabin_class', errors="ignore", inplace=True)           

##### Merge with airline dataframe

In [199]:
# Merge for operating_airline_id
class_price_airline_merged_df = class_flight_price_df.merge(
    airlines_df[['airline_id', 'airline_name']], 
    left_on='operating_airline', 
    right_on='airline_name',
    how='left'
).rename(columns={'airline_id': 'operating_airline_id'})

# Drop redundant column
class_price_airline_merged_df.drop(columns=['airline_name'], inplace=True)

# Merge for marketing_airline_id **without overwriting airline_merged_df**
class_price_airline_merged_df = class_price_airline_merged_df.merge(
    airlines_df[['airline_id', 'airline_name']], 
    left_on='marketing_airline', 
    right_on='airline_name',
    how='left',
    suffixes=('', '_marketing')  # Prevent overwriting columns
).rename(columns={'airline_id': 'marketing_airline_id'})

# Drop redundant column
class_price_airline_merged_df.drop(columns=['airline_name','marketing_airline','operating_airline'], inplace=True)

# Display final DataFrame
class_price_airline_merged_df.head()

,itinerary_id,stop_count,duration_in_hours,one_way_price_candollar,cabin_class_id,operating_airline_id,marketing_airline_id
0,18467-2504010745--31679-1-10968-2504011249,1,2.53,477,cc1,al4,al16
1,18467-2504010745--31679-1-10968-2504011249,1,2.48,477,cc1,al4,al16
2,"18390-2504010700--31954,-31825-2-10968-2504011735",2,0.98,253,cc1,al10,al11
3,18467-2504010615--32385-1-10910-2504011111,1,2.57,581,cc2,al4,al4
4,18467-2504010745--32385-1-10968-2504011249,1,2.53,581,cc2,al4,al4


In [200]:
class_price_airline_merged_df.fillna("al13", inplace=True)

In [201]:
class_price_airline_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11127 entries, 0 to 11126
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   itinerary_id             11127 non-null  object 
 1   stop_count               11127 non-null  int64  
 2   duration_in_hours        11127 non-null  float64
 3   one_way_price_candollar  11127 non-null  int64  
 4   cabin_class_id           11127 non-null  object 
 5   operating_airline_id     11127 non-null  object 
 6   marketing_airline_id     11127 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 608.6+ KB


In [245]:
# Data validation for 
# profiling report for flight_price_data using pandas_profiling.
# Generate a profiling report
profile = ProfileReport(class_price_airline_merged_df, title="flight price report", explorative=True)

# Save the report to an HTML file
profile.to_file("flight_price.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [247]:
class_price_airline_merged_df.to_csv('postgrs_tables/map_flight_price.csv', index=False)
